# Build a Blockchain! - Blockchain Design Exercises

## HW1 - Build a Blockchain in Python

#### Getting Started

CS 198-077 Spring 2020 
HW1-Build a Blockchain in Python

Created by Omkar Shanbhag, Updated by Simon Guo

This notebook is meant to be a short introduction to Blockchain implementations, aimed at helping us take the topics we learn about in fundamentals, and seeing how they translate to code. 

In this notebook we will implement various different aspects of Blockchain technology that we understand including: 

- The Blockchain Data Structure using OOP 
- A proof of work simulation 
- Understanding of the concept of difficulty of finding the next block hash 
- A simulation of multiple miners with varying computational powers 
- A bit of data analytics to see if what we've implemented makes sense 

In [4]:
#Import statements
import hashlib as hasher 
import random as rand
import time 
import datetime as date
import ipyparallel as ipp
import numpy as np
#import matplotlib.pyplot as plt

# Blockchain Data Structure Design

In this section we will define the class "Block" and create an init function that creates a new block given some parameters, as well as a function hash_block, that computes the hash of this block based on its class variables. 

The init function is implemented for you below and creates a block with the following specifications 
- Index --> the index of the block on the chain (zero indexed)
- Timestamp --> time that the block was added on to the chain
- data --> The data the block contains (Usually points to root of merkel tree, but we can fill it with whatever for this)
- previous_hash --> The hash value of the previous block 
- hash --> hash of this block computed using the hash_block function
- nonce --> the variable value that we change to alter the hash output (Default value = 0, irrelevant in this section)

****QUESTION #1: Implement hash_block()****

We need to implement the hash_block() which computes the hash value of this block based on how we initialize it. This function takes in no parameters. 

Desired Functionality:
- Concatenate string representations of all the class variables 
- Computes the SHA256 hash of this concatenation 

-- HINTS --

- use the first import statement!!! 
- initialize a new hasher
- Look into the following words 
    - utf-8 encoding 
    - .update() function 
    - .hexdigest() function
    - In order for the test function below to work properly, create the string concatenation in the following order with no spaces or any other characters in between 
       - index
       - timestamp
       - data
       - previous block hash
       - nonce
     - If you want to do it another order that incorporates everything thats totally fine, but test function wont work!

In [5]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.nonce = nonce #set to zero as default not applicable in first section 
        self.previous_hash = previous_hash
        self.hash = self.hash_block()

    def hash_block(self):
        #Your code for QUESTION 1 Here 
        concatinated = str(self.index).encode() + str(self.timestamp).encode() +self.data.encode() + self.previous_hash.encode() + str(self.nonce).encode()
        hash_object = hasher.sha256(concatinated)
        hex_dig = hash_object.hexdigest()
        return hex_dig
        
        


**TEST YOUR QUESTION 1 CODE HERE**

Run the block of code below

In [6]:
def test_question_1(index, time, data, previous_hash):
    new_block = Block(index, time, data, previous_hash)
    check_string = '2def27922fc1c67254a9cdb0c660b91abf9b135ad38fc13c7c77007448b824a0'
    print_statement = "PASSED!!! Move on to next Question" if str(new_block.hash) == check_string else "FAILED!!! Try Again"
    print(print_statement)
    
time = '2019-10-17 00:37:35.256774'
data = 'Machine Learning Blockchain AI'
previous_hash = '6ffd1464f68ef4aeb385d399244efa19293ba5c842c464a82c02f8256ef71428'
index = 0
    
test_question_1(index, time, data, previous_hash)

PASSED!!! Move on to next Question


**Functions for Building the Chain**

Now that we have our class Block completed, lets figure out how to make a chain out of them. For a chain, we need to first have a function that creates a genesis block, which serves as the first block of our chain, and then create the function next_block() which builds a new block on top of a given block.

create_genesis_block() has been implemented for you. It creates a block with the following specifications. 
- index = 0
- timestamp = Now (whenever the function is being called) 
- data = "Genesis Block"
- previous_hash = "0"

**QUESTION #2: Implement next_block()**

We need to implement the function next_block() which takes in 1 parameter:
- Last_block = an instance of class Block that is the block that we're building our next block on top of 
- nonce = Dont do anything with this right now - just pass it in to the Block that you create using the default 

We want to implement the function so that it returns a new instance of the class Block with the following specifications
- index = index of last_block + 1
- timestamp = Now (whenever the function is being called)
- data = "Hey! I'm block {index}"  (for example block w/ index 5 would have data: "Hey! I'm block 5")
- previous_hash = hash of last_block

In [7]:
#Creates the first block with current time and generic data
def create_genesis_block():
    # Manually construct a block with
    # index zero and arbitrary previous hash
    return Block(0, date.datetime.now(), "Genesis Block", "0")

#Function that creates the next block, given the last block on the chain you want to mine on
def next_block(last_block, nonce=0):
    #Your code for QUESTION 2 here
    this_index = last_block.index + 1
    data = "Hey! I'm block " + str(this_index)
    return Block(this_index, date.datetime.now(), data, last_block.hash)
    

**TEST YOUR QUESTION 2 CODE HERE**

Run the block of code below

In [8]:
def test_question_2(genesis_block):
    block_1 = next_block(genesis_block)
    if block_1.index == 1 and block_1.data == "Hey! I'm block 1" and block_1.previous_hash == genesis_block.hash and str(type(block_1.timestamp)) == "<class 'datetime.datetime'>":
        print("PASSED!!! Move on to next part" )
    else:
        print("FAILED!!! Try again :(")
    

genesis_block = create_genesis_block()
test_question_2(genesis_block)

PASSED!!! Move on to next part


**Spinning up a Chain**

Now that we've created the data structure as well as the functions needed to create the chain, lets see how spinning up an actual instance of this would work. 

Below we initialize three different variables 
- blockchain - this is a python list which we initialize with one block inside (the genesis block) 
- previous_block - this points to our genesis block (since it references the first element in blockchain)
- num_blocks - this specifies the number of additional blocks we want to add to our chain 

**QUESTION 3 complete_chain()**

We want to complete the implementation of the function complete_chain(). This function takes in three inputs, which correspond to the initializations that we made. It returns nothing, however by the time we are done running it, the list 'blockchain' that we initialized earlier has been turned into an array of length num_blocks + 1 in which each element is an instance of class Block and each element's self.previous_hash == the previous element's self.hash. Therefore we have created our own mini blockchain!! 

The for loop and the print statements of complete_chain have been implemented for you, you need to add the statements that ***create a new block on top of previous_block, add it to the block chain, and edit previous block so that the for loop can continue correctly*** 

****HINT **** --> Literally just do all the things listed in the section immediately above in order 

If it works out, you should get the same number of print statements as num_blocks

In [9]:
# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]

#Create our initial reference to previous block which points to the genesis block
previous_block = blockchain[0]

# How many blocks should we add to the chain after the genesis block
num_blocks = 20

def complete_chain(num_blocks, blockchain, previous_block):
    # Add blocks to the chain
    for i in range(0, num_blocks):
        #Your code for QUESTION 3 Here
        block_to_add = next_block(previous_block)
        blockchain.append(block_to_add)
        previous_block = block_to_add

        
        #Your code for QUESTION 3 ends Here
        # Tell everyone about it!
        print("Block #{} has been added to the blockchain!".format(block_to_add.index))
        print("Hash: {}\n".format(block_to_add.hash))

complete_chain(num_blocks, blockchain, previous_block)

Block #1 has been added to the blockchain!
Hash: 06c25522696126e14e68be2a4ad3ab702c0e74cb4ed0958c99d5eb95fbeca25e

Block #2 has been added to the blockchain!
Hash: 80f2fe394b3dfe20ddb1e7bb347216b1a3edc490d9594ce64295ccfb05fe5f14

Block #3 has been added to the blockchain!
Hash: 85e94420dd26e4e4f23096c1357300f82efd34d3b7466febecd4523754ff33d6

Block #4 has been added to the blockchain!
Hash: bdeaa6170bbd27690a44a5487b10e58915df5ec09fd53cbdb5b913107a1c33e6

Block #5 has been added to the blockchain!
Hash: d91dcd80a8138401f6b3eb01ace9f61cea062d903d7b62c7eb5780076487d42a

Block #6 has been added to the blockchain!
Hash: 44318ae2af1464800db35f096c3fea15048686d34005595b6294e7a4dc2f17e2

Block #7 has been added to the blockchain!
Hash: 9e2533f6c8e930d802dd615f1c1a334e36ec966db9cdfb753cc1c310c6f8c50b

Block #8 has been added to the blockchain!
Hash: 80c9b1bb701f403584233e5d0802e87922a3fcd11dd53260f37b860a7564e68a

Block #9 has been added to the blockchain!
Hash: b5eb251ed1528088cb20fb580e9aab6

**TEST YOUR QUESTION 3 CODE HERE**

Run the block of code below

In [10]:
def test_question_3(blockchain, num_blocks):
    correct = True
    if len(blockchain) != num_blocks + 1:
        correct = False
    for i in range(len(blockchain)-1):
        if blockchain[i + 1].previous_hash != blockchain[i].hash:
            correct = False
            break
    print_statement = "PASSED!!! Move on to the next Part" if correct else "FAILED!!! Try Again :("
    print(print_statement)

test_question_3(blockchain, num_blocks)

PASSED!!! Move on to the next Part


# Proof of Work Simulation

In this section we will be doing a simulation of the proof-of-work consensus mechanism that the Bitcoin Blockchain (among others) uses. Let us define some of the concepts that we will be dealing with in this section. 

**The Nonce** --> Randomly generated value that we add to our concatenation of our block to add variance to our hashes

**Difficulty** --> Specified by the network (in theory, here it is specified by us). Defines the number of hashes that are valid out of all possible values. Higher difficulty indicates a lower number of valid hashes. 

**QUESTION 4: generate_nonce(), generate_difficulty_bound(), find_next_block()**

**Part 1 - Description** 

The functions **generate_nonce()** and **generate_difficulty_bound()** have been implemented below for you. The first part of this question involves reading through, them understanding them completely, and writing a quick 1 - 2 line summary of what they are supposed to do and how they are implemented (write it in the space given, 2 cells below).

**Part 2 - Implementation**

The second part of this question is to complete the implementation of find_next_block(). This function's purpose is to try different blocks with the same data, index etc. but different nonces that satisfy the difficulty metric specified. The difficulty bound has already been generated for you and the first block has been created. You have to complete the implementation so that: 
- You find a nonce such that the hash of the block is less than the difficulty bound. 
- All data from new block has to be the same (including the timestamp, for simplicity purposes) and the only thing changing is the nonce 
- For our learning purposes, increment the hashes_tried parameter appropriately everytime you try a hash so that we can see the number of hashes tried and how this correlates to our difficulty value. 

**HINTS**
- How do you turn a hash string into its hex prepresentation that you can compare???
- if your implementation is working, you should notice something interesting about all the hashes of the blocks that you are getting when you run the code cell labeled "Create Proof of work Blockchain"**



**you can test question 4 in the a few cells below this one, after running all the cells in between**

In [41]:
import time

def generate_nonce(length=20):
    return ''.join([str(rand.randint(0, 9)) for i in range(length)])

def generate_difficulty_bound(difficulty=1):
    diff_str = ""
    for i in range(difficulty):
        diff_str += '0'
    for i in range(64 - difficulty):
        diff_str += 'F'
    diff_str = "0x" + diff_str  # "0x" needs to be added at the front to specify that it is a hex representation
    return(int(diff_str, 16))  # Specifies that we want to create an integer of base 16 (as opposed to the default base 10)

#Given a previous block and a difficulty metric, finds a nonce that results in a lower hash value
def find_next_block(last_block, difficulty, nonce_length):
    difficulty_bound = generate_difficulty_bound(difficulty)
    start = time.process_time() 
    new_block = next_block(last_block)
    hashes_tried = 1 
    #Your code for QUESTION 4 Starts here 
    
    #convert hash block to hex representation
    trial = int(new_block.hash, 16)
    
    #breaks when the trial is less than of equal to the bound
    while trial > difficulty_bound:
        
        #convert new hash block to hex representation
        test_block = Block(new_block.index, new_block.timestamp, new_block.data, new_block.previous_hash, generate_nonce(nonce_length))
        trial = int(test_block.hash, 16)
        hashes_tried += 1
        
    new_block = test_block

    #Your code for QUESTION 4 Ends here 
    time_taken = time.process_time() - start
    return(time_taken, hashes_tried, new_block)

In [42]:
print(generate_difficulty_bound(2))

452312848583266388373324160190187140051835877600158453279131187530910662655


**QUESTION 4 Description Section**

Describe the following functions: 

- generate_nonce():         
generates a nonce, a 20 line string of random integers

- generate_difficulty_bound():       
fils up 0 up to the difficulty leve of the diff_str. then fills up the rest of diff_str with 'F'. Then '0x' is added to the head. Then the diff_str is returned in base-16 hexidecimals.


The cell below creates our proof of work blockchain in a similar way that we do in the earlier section. 

Some initializations:
- blockchain_pow: Our new python list that signifies our proof of work blockchain, with the genesis block inside
- previous_block: The first block to use as previous block to build upon
- num_blocks: number of additional blocks to add to teh chain
- difficulty: difficulty of the network 
- nonce_length: length of the randomly generated nonce

### Create Proof of Work Blockchain 

In [43]:
# Create the blockchain and add the genesis block
blockchain_pow = [create_genesis_block()]

#Create our initial reference to previous block which points to the genesis block
previous_block = blockchain_pow[0]

# How many blocks should we add to the chain after genesis block
num_blocks = 20

#magnitude of difficulty of hash - number of zeroes that must be in the beginning of the hash
difficulty = 3

#length of nonce that will be generated and added
nonce_length = 20

# Add blocks to the chain based on difficulty with nonces of length nonce_length
def create_pow_blockchain(num_blocks, difficulty, blockchain_pow, previous_block, nonce_length, print_data=1):
    hash_array = []
    time_array = []
    for i in range(0, num_blocks):
        time_taken, hashes_tried, block_to_add = find_next_block(previous_block, difficulty, nonce_length)
        blockchain_pow.append(block_to_add)
        previous_block = block_to_add
        hash_array.append(hashes_tried)
        time_array.append(time_taken)
        # Tell everyone about it!
        if print_data:
            print("Block #{} has been added to the blockchain!".format(block_to_add.index))
            print("{} Hashes Tried!".format(hashes_tried))
            print("Time taken to find block: {}".format(time_taken))
            print("Hash: {}\n".format(block_to_add.hash))
    return(hash_array, time_array)

hash_array, time_array = create_pow_blockchain(num_blocks, difficulty, blockchain_pow, previous_block, nonce_length)

Block #1 has been added to the blockchain!
3632 Hashes Tried!
Time taken to find block: 0.359375
Hash: 0001ba0338a7566777f3c679a2ce095fb4c2d845c613cc86260eaebb45199b35

Block #2 has been added to the blockchain!
216 Hashes Tried!
Time taken to find block: 0.03125
Hash: 00012240d683c6d253625e4741099f0bfc43ff4a9a62eb5f3136f78a7e7b56a7

Block #3 has been added to the blockchain!
353 Hashes Tried!
Time taken to find block: 0.03125
Hash: 00095dfe304bda076b8db6426ecae64f60b8d025c33ba1abd87e6ac1d3ea1775

Block #4 has been added to the blockchain!
2271 Hashes Tried!
Time taken to find block: 0.171875
Hash: 000ed04c79e785da45163ee0072aa93e5c16d2973d66ae0a80ad7e14fe30ea0a

Block #5 has been added to the blockchain!
2410 Hashes Tried!
Time taken to find block: 0.265625
Hash: 000070fc0a13cf202638c6883337e1192a74adfee909dab3d509f2aa9cd8e762

Block #6 has been added to the blockchain!
5484 Hashes Tried!
Time taken to find block: 0.484375
Hash: 000dd99de3603b43ce5f067200f1c6f9733e783e8b052de042dab008

**TEST YOUR QUESTION 4 CODE HERE**

Run the block of code below

In [44]:
def test_question_4(blockchain_pow, num_blocks):
    correct = True
    bound = generate_difficulty_bound(difficulty)
    if len(blockchain_pow) != num_blocks + 1:
        correct = False
    for i in range(len(blockchain_pow) - 1):
        if blockchain_pow[i + 1].previous_hash != blockchain_pow[i].hash:
            correct = False
            break
        if int(blockchain_pow[i + 1].hash, 16) > bound:
            correct = False
            print('why')
            break
    print_statement = "PASSED!!! Move on to the next Part" if correct else "FAILED!!! Try Again :("
    print(print_statement)
            
test_question_4(blockchain_pow, num_blocks)

PASSED!!! Move on to the next Part


Once you have passed the above test case, you can do things like play around with the difficulties and nonce lengths and seeing that happens 

# Note: The sections below are optional but super cool!

# Distributed Network + Compute Power Simulation

In this section, instead of simulating a single node that carries out all the proof of work calculations, we will create a bunch of different nodes that will all compete to find different blocks. These nodes will all have different compute powers that we will be able to specify. Below we will define the MinerNodeNaive class which has an init function to create an instance, as well as a try_hash function that allows it to try a hash and see if it works. 

In [15]:
#Naive miner class that races with other miners to see who can get a certain number of blocks first 
class MinerNodeNaive: 
    def __init__(self, name, compute):
        self.name = name 
        self.compute = compute
    
    def try_hash(self, diff_value, chain):
        last_block = chain[-1]
        difficulty = generate_difficulty_bound(diff_value)
        date_now = date.datetime.now()
        this_index = last_block.index + 1
        this_timestamp = date_now
        this_data = "Hey! I'm block " + str(this_index)
        this_hash = last_block.hash
        new_block = Block(this_index, this_timestamp, this_data, this_hash)
        if int(new_block.hash, 16) < difficulty:
            chain.append(new_block)
            # Tell everyone about it!
            print("Block #{} has been added to the blockchain!".format(new_block.index))
            print("Block found by: {}".format(self.name))
            print("Hash: {}\n".format(new_block.hash))
        

**Question #5: Describe the Following Cells**

In the next 4 cells I try to simulate a bunch of different miners with different compute powers. However this isn't completely indicative of how a real system works, and is limited by what I had to work with. Read through the code, and run all of the cells and explain what is happening in a paragrpah. Also explain how this is different from real world vanilla proof of work systems and how this simulation isn't completely accurate of what's actually happening while nodes race to find the next block. There is a cell at the bottom where you can enter your description.  

In [16]:
#Initialize multiple miners on the network 
berkeley_Miner = MinerNodeNaive("Berkeley Miner", 10)
stanford_Miner = MinerNodeNaive("Stanford Miner", 5)
MIT_Miner = MinerNodeNaive("MIT Miner", 2)
UCLA_Miner = MinerNodeNaive("UCLA Miner", 1)

miner_array = [berkeley_Miner, stanford_Miner, MIT_Miner, UCLA_Miner]

In [17]:
def create_compute_simulation(miner_array):
    compute_array = []
    for miner in miner_array:
        for i in range(miner.compute):
            compute_array.append(miner.name)
    return(compute_array)

compute_simulation_array = create_compute_simulation(miner_array)
rand.shuffle(compute_simulation_array)

In [18]:
chain_length = 20
blockchain_distributed = [create_genesis_block()]
genesis_block_dist = blockchain_distributed[0]
chain_difficulty = [rand.randint(2,4) for i in range(chain_length)]

In [19]:
for i in range(len(chain_difficulty)): 
    while len(blockchain_distributed) < i + 2:
        next_miner_str = rand.sample(compute_simulation_array, 1)[0] 
        next_miner = berkeley_Miner #random default (go bears)
        for miner in miner_array:
            if next_miner_str == miner.name:
                next_miner = miner
        next_miner.try_hash(chain_difficulty[i], blockchain_distributed)
        

Block #1 has been added to the blockchain!
Block found by: Berkeley Miner
Hash: 0000d63be2bbdae561406c8e6241ace2e940e44f092e2fe697a51f627c2ea973

Block #2 has been added to the blockchain!
Block found by: MIT Miner
Hash: 0002d2227edcfbdfd57d4d30b9f1799a242b8981eb5ee1148e0232dab879c68c

Block #3 has been added to the blockchain!
Block found by: Berkeley Miner
Hash: 00007270b2a3bc01d50d53bb27b9cf274fc8a4af8ec08b85616da477d006a7ed

Block #4 has been added to the blockchain!
Block found by: Berkeley Miner
Hash: 000fe78eae07257e9199bd3d7c61fc062015174b873b017cf191dda4547d8a13

Block #5 has been added to the blockchain!
Block found by: Stanford Miner
Hash: 00224ed4df1ddea6c399fc1e0ffceb1762094db46a0b80e6462fe3d82d6b48f9

Block #6 has been added to the blockchain!
Block found by: Stanford Miner
Hash: 002b5ddac3a3a11d5131e2cf647239108fe4245afdd55f52771a52b5e285530f

Block #7 has been added to the blockchain!
Block found by: Stanford Miner
Hash: 00c6add2150058aeea5ab2a297bf49ef1d0fbba4f3a747fdd

**Question 5 Description Here**

- Description: **#####Your description here####**

# Blockchain Data Analytics

Here we will use data analytics to analyze the number of hashes we need to try before we find a valid hash, given a difficulty level. Look through the code briefly (in depth is not necessary), run all of the cells, and answer the question at the bottom. (difficulty level 3 takes a bit of time so dont be alarmed if this next cell takes a bit of time to run). 

In [20]:
blockchain = [create_genesis_block()]
previous_block = blockchain[0]
num_blocks = 10

#3 different types of difficulty to analyze
difficulty_0 = 1
difficulty_1 = 2
difficulty_2 = 3
difficulty_3 = 4

nonce_length = 20

hash_array_0, time_array_0 = create_pow_blockchain(num_blocks, difficulty_0, blockchain, previous_block, nonce_length, 0)
print("Difficulty Level: {} complete".format(difficulty_0))
hash_array_1, time_array_1 = create_pow_blockchain(num_blocks, difficulty_1, blockchain, previous_block, nonce_length, 0)
print("Difficulty Level: {} complete".format(difficulty_1))
hash_array_2, time_array_2 = create_pow_blockchain(num_blocks, difficulty_2, blockchain, previous_block, nonce_length, 0)
print("Difficulty Level: {} complete".format(difficulty_2))
hash_array_3, time_array_3 = create_pow_blockchain(num_blocks, difficulty_3, blockchain, previous_block, nonce_length, 0)
print("Difficulty Level: {} complete".format(difficulty_3))

Difficulty Level: 1 complete
Difficulty Level: 2 complete
Difficulty Level: 3 complete
Difficulty Level: 4 complete


In [21]:
mean_arr_hash = [np.mean(hash_array_0), np.mean(hash_array_1), np.mean(hash_array_2), np.mean(hash_array_3)]
mean_arr_time = [np.mean(time_array_0), np.mean(time_array_1), np.mean(time_array_2), np.mean(time_array_3)]

In [22]:
plt.plot(mean_arr_hash)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.plot(mean_arr_time)
plt.show()

In [ ]:
diff_factor_1 = np.mean(hash_array_1)/np.mean(hash_array_0)
diff_factor_2 = np.mean(hash_array_2)/np.mean(hash_array_1)
diff_factor_3 = np.mean(hash_array_3)/np.mean(hash_array_2)
print("Factor of difficulty increase from 1 to 2: {}".format(diff_factor_1))
print("Factor of difficulty increase from 2 to 3: {}".format(diff_factor_2))
print("Factor of difficulty increase from 3 to 4: {}".format(diff_factor_3))

**Question 6: This one is simple**

Look at the factor of difficulty increase from each level to the subsequent level, printed from the cell above.

What whould the factor of difficulty increase be for each level: **Your answer Here**

**Check Off**

If you have filled out all the skeleton code and put down your answers for the questions, let any of the instructor know and we will check you off!

If you cannot finish it this class, you have until next week's class to finish check off and earn points.
